In [1]:
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime,timezone,timedelta
import pytz #time zone data

In [2]:
def unix_to_aus(time):
    
    """this function convert UNIX date time to Austrelia  date time and output will be string"""
    
    time_int = int(time) #get integer value
    
    time_zone = timezone(timedelta(seconds=36000)) # time zone of Austrelia 
    
    aus_time = datetime.fromtimestamp(time_int, tz = time_zone).strftime('%Y-%m-%d %H:%M:%S')
    #aus_time = datetime.fromtimestamp(time_int, tz = time_zone)
    
    return aus_time

In [3]:
def find_solar_seconds(Sunrise,Sunset):
    
    """ this function got two datetime vectors and return the time diffrence between each two elements in seconds as an array """
    
    Timedelta_list = [] # tiem diffrence list
    
    for i in range(len(Sunrise)):
        
        Timedelta_str_whole = Sunset[i] - Sunrise[i] # output example: Timedelta('0 days 12:17:55')
        Timedelta_str_time = str(Timedelta_str_whole).split()[2] # output example : '12:17:55'
        Timedelta_str_num = Timedelta_str_time.split(':') # output example: ['12', '17', '55'] 
        Timedelta_num = [int(i) for i in Timedelta_str_num] # output example: [12, 17, 55]
        
        Timedelta_sconds = Timedelta_num[0]*3600 + Timedelta_num[1]*60 + Timedelta_num[2] # output example: 44275
        
        Timedelta_list.append(Timedelta_sconds)
        
    return np.array(Timedelta_list)

### Load and understand data

In [3]:
grid_point_dataset = pd.read_csv("../spatial/data/csv/grid70m_points.csv")

In [5]:
grid_point_Weather_data = pd.DataFrame(columns=["Time","Id","Longitude", "Latitude","Tempreture", "Humidity","Wind speed","Weather ID", "Weather ID group", "Weather ID description", "Sunrise", "Sunset"])

In [6]:
grid_point_dataset.head()

id   Longitude   Latitude  Probability
0   1  143.251973 -35.083201     0.347612
1   2  143.251973 -35.084201     0.867893
2   3  143.251973 -35.085201     0.443985
3   4  143.251973 -35.086201     0.055013
4   5  143.251973 -35.087201     0.016234

In [7]:
grid_point_dataset.isnull().sum()

id             0
Longitude      0
Latitude       0
Probability    0
dtype: int64

In [8]:
grid_point_Weather_data.head()

Empty DataFrame
Columns: [Time, Id, Longitude, Latitude, Tempreture, Humidity, Wind speed, Weather ID, Weather ID group, Weather ID description, Sunrise, Sunset]
Index: []

###  Call API and get gata into data frame

In [5]:
longitudes = np.array(grid_point_dataset["Longitude"])
latitudes = np.array(grid_point_dataset["Latitude"])

In [10]:
longitudes

array([143.25197304, 143.25197304, 143.25197304, ..., 143.31697304,
       143.31697304, 143.31697304])

In [11]:
int(len(latitudes)/4)

990

In [14]:
srt_time  = datetime.now()
piangil_timezone = pytz.timezone('Australia/Sydney')

for i in range(int(len(latitudes)/4)): # contralls the amount of the data
    
    srt_time_point  = datetime.now()
    #get the lat long coordinates
    lat = latitudes[4*i] 
    long = longitudes[4*i]
    
    #API url
    url = "https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid=8ee842d65cf08ec205365865e3d53348&units=metric".format(lat,long)
    
    piangil_time = datetime.now(piangil_timezone) #get time in Australia for data set
    
    #get data form API as json data
    res = requests.get(url)
    data = res.json()
    
    # create the data list that we want from the json data 
    data_vec = [piangil_time, 4*i+1, long, lat, data["main"]["temp"], data["main"]["humidity"], data["wind"]["speed"], data["weather"][0]["id"], data["weather"][0]["main"], data["weather"][0]["description"], unix_to_aus(data["sys"]["sunrise"]), unix_to_aus(data["sys"]["sunset"])]
    data_vec_1 = [piangil_time, 4*i+1+1, longitudes[4*i+1], latitudes[4*i+1], data["main"]["temp"], data["main"]["humidity"], data["wind"]["speed"], data["weather"][0]["id"], data["weather"][0]["main"], data["weather"][0]["description"], unix_to_aus(data["sys"]["sunrise"]), unix_to_aus(data["sys"]["sunset"])]
    data_vec_2 = [piangil_time, 4*i+2+1, longitudes[4*i+2], latitudes[4*i+2], data["main"]["temp"], data["main"]["humidity"], data["wind"]["speed"], data["weather"][0]["id"], data["weather"][0]["main"], data["weather"][0]["description"], unix_to_aus(data["sys"]["sunrise"]), unix_to_aus(data["sys"]["sunset"])]
    data_vec_3 = [piangil_time, 4*i+3+1, longitudes[4*i+3], latitudes[4*i+3], data["main"]["temp"], data["main"]["humidity"], data["wind"]["speed"], data["weather"][0]["id"], data["weather"][0]["main"], data["weather"][0]["description"], unix_to_aus(data["sys"]["sunrise"]), unix_to_aus(data["sys"]["sunset"])]
    
    #update the data frame
    grid_point_Weather_data.loc[4*i] = data_vec
    grid_point_Weather_data.loc[4*i+1] = data_vec_1
    grid_point_Weather_data.loc[4*i+2] = data_vec_2
    grid_point_Weather_data.loc[4*i+3] = data_vec_3
    
    
    
    time.sleep(0.1)
    end_time_point  = datetime.now()
    print(f"step {i+1} is completed! and taken {end_time_point-srt_time_point} time to complete")
    
    
    
end_time = datetime.now()
total_execution_time = end_time-srt_time
print(f"the programe take: {total_execution_time} to complete" )          

step 1 is completed! and taken 0:00:00.885062 time to complete
step 2 is completed! and taken 0:00:01.066986 time to complete
step 3 is completed! and taken 0:00:00.928121 time to complete
step 4 is completed! and taken 0:00:01.043046 time to complete
step 5 is completed! and taken 0:00:01.031879 time to complete
step 6 is completed! and taken 0:00:00.957233 time to complete
step 7 is completed! and taken 0:00:00.898721 time to complete
step 8 is completed! and taken 0:00:00.874400 time to complete
step 9 is completed! and taken 0:00:01.017258 time to complete
step 10 is completed! and taken 0:00:01.112369 time to complete
step 11 is completed! and taken 0:00:01.013407 time to complete
step 12 is completed! and taken 0:00:00.984553 time to complete
step 13 is completed! and taken 0:00:00.962132 time to complete
step 14 is completed! and taken 0:00:00.949355 time to complete
step 15 is completed! and taken 0:00:00.937889 time to complete
step 16 is completed! and taken 0:00:01.038969 ti

step 129 is completed! and taken 0:00:00.933689 time to complete
step 130 is completed! and taken 0:00:01.133998 time to complete
step 131 is completed! and taken 0:00:01.093928 time to complete
step 132 is completed! and taken 0:00:01.152028 time to complete
step 133 is completed! and taken 0:00:01.075908 time to complete
step 134 is completed! and taken 0:00:01.063523 time to complete
step 135 is completed! and taken 0:00:00.985766 time to complete
step 136 is completed! and taken 0:00:00.946049 time to complete
step 137 is completed! and taken 0:00:01.099156 time to complete
step 138 is completed! and taken 0:00:00.926182 time to complete
step 139 is completed! and taken 0:00:00.972084 time to complete
step 140 is completed! and taken 0:00:01.039760 time to complete
step 141 is completed! and taken 0:00:01.174757 time to complete
step 142 is completed! and taken 0:00:00.986239 time to complete
step 143 is completed! and taken 0:00:01.006441 time to complete
step 144 is completed! an

step 256 is completed! and taken 0:00:00.949347 time to complete
step 257 is completed! and taken 0:00:00.864491 time to complete
step 258 is completed! and taken 0:00:01.027042 time to complete
step 259 is completed! and taken 0:00:01.687210 time to complete
step 260 is completed! and taken 0:00:00.934386 time to complete
step 261 is completed! and taken 0:00:00.922092 time to complete
step 262 is completed! and taken 0:00:01.024076 time to complete
step 263 is completed! and taken 0:00:01.059927 time to complete
step 264 is completed! and taken 0:00:00.906525 time to complete
step 265 is completed! and taken 0:00:01.468411 time to complete
step 266 is completed! and taken 0:00:01.239129 time to complete
step 267 is completed! and taken 0:00:00.981516 time to complete
step 268 is completed! and taken 0:00:00.878091 time to complete
step 269 is completed! and taken 0:00:00.922822 time to complete
step 270 is completed! and taken 0:00:01.292140 time to complete
step 271 is completed! an

step 383 is completed! and taken 0:00:01.529038 time to complete
step 384 is completed! and taken 0:00:01.056720 time to complete
step 385 is completed! and taken 0:00:01.045661 time to complete
step 386 is completed! and taken 0:00:01.054178 time to complete
step 387 is completed! and taken 0:00:01.017138 time to complete
step 388 is completed! and taken 0:00:01.077030 time to complete
step 389 is completed! and taken 0:00:00.936328 time to complete
step 390 is completed! and taken 0:00:01.037917 time to complete
step 391 is completed! and taken 0:00:01.049297 time to complete
step 392 is completed! and taken 0:00:00.937828 time to complete
step 393 is completed! and taken 0:00:01.341605 time to complete
step 394 is completed! and taken 0:00:00.982848 time to complete
step 395 is completed! and taken 0:00:01.049800 time to complete
step 396 is completed! and taken 0:00:00.968365 time to complete
step 397 is completed! and taken 0:00:01.015164 time to complete
step 398 is completed! an

step 510 is completed! and taken 0:00:00.970535 time to complete
step 511 is completed! and taken 0:00:02.995373 time to complete
step 512 is completed! and taken 0:00:01.280529 time to complete
step 513 is completed! and taken 0:00:01.620611 time to complete
step 514 is completed! and taken 0:00:01.217542 time to complete
step 515 is completed! and taken 0:00:01.578106 time to complete
step 516 is completed! and taken 0:00:01.178793 time to complete
step 517 is completed! and taken 0:00:01.659150 time to complete
step 518 is completed! and taken 0:00:01.020457 time to complete
step 519 is completed! and taken 0:00:01.195992 time to complete
step 520 is completed! and taken 0:00:01.411943 time to complete
step 521 is completed! and taken 0:00:01.237136 time to complete
step 522 is completed! and taken 0:00:01.213105 time to complete
step 523 is completed! and taken 0:00:01.130800 time to complete
step 524 is completed! and taken 0:00:01.159724 time to complete
step 525 is completed! an

step 637 is completed! and taken 0:00:00.971774 time to complete
step 638 is completed! and taken 0:00:01.069512 time to complete
step 639 is completed! and taken 0:00:01.031885 time to complete
step 640 is completed! and taken 0:00:00.942812 time to complete
step 641 is completed! and taken 0:00:01.064278 time to complete
step 642 is completed! and taken 0:00:01.086435 time to complete
step 643 is completed! and taken 0:00:01.012913 time to complete
step 644 is completed! and taken 0:00:01.065454 time to complete
step 645 is completed! and taken 0:00:00.882828 time to complete
step 646 is completed! and taken 0:00:01.078637 time to complete
step 647 is completed! and taken 0:00:01.327386 time to complete
step 648 is completed! and taken 0:00:01.591292 time to complete
step 649 is completed! and taken 0:00:01.235316 time to complete
step 650 is completed! and taken 0:00:01.000945 time to complete
step 651 is completed! and taken 0:00:01.040985 time to complete
step 652 is completed! an

step 764 is completed! and taken 0:00:01.026172 time to complete
step 765 is completed! and taken 0:00:01.020648 time to complete
step 766 is completed! and taken 0:00:00.993613 time to complete
step 767 is completed! and taken 0:00:00.958557 time to complete
step 768 is completed! and taken 0:00:01.560744 time to complete
step 769 is completed! and taken 0:00:00.993725 time to complete
step 770 is completed! and taken 0:00:00.975813 time to complete
step 771 is completed! and taken 0:00:01.203509 time to complete
step 772 is completed! and taken 0:00:00.950099 time to complete
step 773 is completed! and taken 0:00:01.058757 time to complete
step 774 is completed! and taken 0:00:00.921728 time to complete
step 775 is completed! and taken 0:00:00.935391 time to complete
step 776 is completed! and taken 0:00:00.932130 time to complete
step 777 is completed! and taken 0:00:01.059351 time to complete
step 778 is completed! and taken 0:00:00.941159 time to complete
step 779 is completed! an

step 891 is completed! and taken 0:00:00.835727 time to complete
step 892 is completed! and taken 0:00:01.041412 time to complete
step 893 is completed! and taken 0:00:01.084188 time to complete
step 894 is completed! and taken 0:00:01.010915 time to complete
step 895 is completed! and taken 0:00:01.177303 time to complete
step 896 is completed! and taken 0:00:00.897694 time to complete
step 897 is completed! and taken 0:00:01.045689 time to complete
step 898 is completed! and taken 0:00:00.924483 time to complete
step 899 is completed! and taken 0:00:00.974667 time to complete
step 900 is completed! and taken 0:00:00.876496 time to complete
step 901 is completed! and taken 0:00:01.012772 time to complete
step 902 is completed! and taken 0:00:01.135423 time to complete
step 903 is completed! and taken 0:00:01.149091 time to complete
step 904 is completed! and taken 0:00:00.959465 time to complete
step 905 is completed! and taken 0:00:00.965239 time to complete
step 906 is completed! an

In [15]:
grid_point_Weather_data.head()

Time  Id   Longitude   Latitude  Tempreture  \
0 2023-11-28 22:53:55.606668+11:00   1  143.251973 -35.083201       16.92   
1 2023-11-28 22:53:55.606668+11:00   2  143.251973 -35.084201       16.92   
2 2023-11-28 22:53:55.606668+11:00   3  143.251973 -35.085201       16.92   
3 2023-11-28 22:53:55.606668+11:00   4  143.251973 -35.086201       16.92   
4 2023-11-28 22:53:56.491730+11:00   5  143.251973 -35.087201       16.93   

   Humidity  Wind speed  Weather ID Weather ID group Weather ID description  \
0        89        5.48         500             Rain             light rain   
1        89        5.48         500             Rain             light rain   
2        89        5.48         500             Rain             light rain   
3        89        5.48         500             Rain             light rain   
4        89        5.45         500             Rain             light rain   

               Sunrise               Sunset  
0  2023-11-28 05:07:11  2023-11-28 19:22:34  
1  2023-11-28 05:07:11  2023-11-28 19:22:34  
2  2023-11-28 05:07:11  2023-11-28 19:22:34  
3  2023-11-28 05:07:11  2023-11-28 19:22:34  
4  2023-11-28 05:07:11  2023-11-28 19:22:35

### Preprocessing and feature engineering

In [16]:
# convert string type timedate data in to dateime type data
grid_point_Weather_data['Sunrise'] = pd.to_datetime(grid_point_Weather_data['Sunrise'], format='%Y-%m-%d %H:%M:%S')
grid_point_Weather_data['Sunset'] = pd.to_datetime(grid_point_Weather_data['Sunset'], format='%Y-%m-%d %H:%M:%S')

In [17]:
grid_point_Weather_data.head(10)

Time  Id   Longitude   Latitude  Tempreture  \
0 2023-11-28 22:53:55.606668+11:00   1  143.251973 -35.083201       16.92   
1 2023-11-28 22:53:55.606668+11:00   2  143.251973 -35.084201       16.92   
2 2023-11-28 22:53:55.606668+11:00   3  143.251973 -35.085201       16.92   
3 2023-11-28 22:53:55.606668+11:00   4  143.251973 -35.086201       16.92   
4 2023-11-28 22:53:56.491730+11:00   5  143.251973 -35.087201       16.93   
5 2023-11-28 22:53:56.491730+11:00   6  143.251973 -35.088201       16.93   
6 2023-11-28 22:53:56.491730+11:00   7  143.251973 -35.089201       16.93   
7 2023-11-28 22:53:56.491730+11:00   8  143.251973 -35.090201       16.93   
8 2023-11-28 22:53:57.558716+11:00   9  143.251973 -35.091201       16.93   
9 2023-11-28 22:53:57.558716+11:00  10  143.251973 -35.092201       16.93   

   Humidity  Wind speed  Weather ID Weather ID group Weather ID description  \
0        89        5.48         500             Rain             light rain   
1        89        5.48         500             Rain             light rain   
2        89        5.48         500             Rain             light rain   
3        89        5.48         500             Rain             light rain   
4        89        5.45         500             Rain             light rain   
5        89        5.45         500             Rain             light rain   
6        89        5.45         500             Rain             light rain   
7        89        5.45         500             Rain             light rain   
8        89        5.45         500             Rain             light rain   
9        89        5.45         500             Rain             light rain   

              Sunrise              Sunset  
0 2023-11-28 05:07:11 2023-11-28 19:22:34  
1 2023-11-28 05:07:11 2023-11-28 19:22:34  
2 2023-11-28 05:07:11 2023-11-28 19:22:34  
3 2023-11-28 05:07:11 2023-11-28 19:22:34  
4 2023-11-28 05:07:11 2023-11-28 19:22:35  
5 2023-11-28 05:07:11 2023-11-28 19:22:35  
6 2023-11-28 05:07:11 2023-11-28 19:22:35  
7 2023-11-28 05:07:11 2023-11-28 19:22:35  
8 2023-11-28 05:07:11 2023-11-28 19:22:35  
9 2023-11-28 05:07:11 2023-11-28 19:22:35

#### Feature1: solar activation time 

In [18]:
Sunrise = grid_point_Weather_data["Sunrise"]
Sunset = grid_point_Weather_data["Sunset"]

# find the solar activation time
time_diffrence_arr = find_solar_seconds(Sunrise,Sunset)
time_diffrence_arr

array([51323, 51323, 51323, ..., 51338, 51338, 51338])

In [19]:
grid_point_Weather_data["Solar activation sconds"] = time_diffrence_arr

In [20]:
grid_point_Weather_data.head()

Time  Id   Longitude   Latitude  Tempreture  \
0 2023-11-28 22:53:55.606668+11:00   1  143.251973 -35.083201       16.92   
1 2023-11-28 22:53:55.606668+11:00   2  143.251973 -35.084201       16.92   
2 2023-11-28 22:53:55.606668+11:00   3  143.251973 -35.085201       16.92   
3 2023-11-28 22:53:55.606668+11:00   4  143.251973 -35.086201       16.92   
4 2023-11-28 22:53:56.491730+11:00   5  143.251973 -35.087201       16.93   

   Humidity  Wind speed  Weather ID Weather ID group Weather ID description  \
0        89        5.48         500             Rain             light rain   
1        89        5.48         500             Rain             light rain   
2        89        5.48         500             Rain             light rain   
3        89        5.48         500             Rain             light rain   
4        89        5.45         500             Rain             light rain   

              Sunrise              Sunset  Solar activation sconds  
0 2023-11-28 05:07:11 2023-11-28 19:22:34                    51323  
1 2023-11-28 05:07:11 2023-11-28 19:22:34                    51323  
2 2023-11-28 05:07:11 2023-11-28 19:22:34                    51323  
3 2023-11-28 05:07:11 2023-11-28 19:22:34                    51323  
4 2023-11-28 05:07:11 2023-11-28 19:22:35                    51324

In [21]:
# create a date column as first column
date_column = grid_point_Weather_data["Sunrise"].apply(lambda x: ((str(x)).split())[0])
grid_point_Weather_data.insert(0, "Date",date_column)

# update the Time column
grid_point_Weather_data["Time"] = grid_point_Weather_data["Time"].apply(lambda x: (str(x)).split()[1][:11])

In [22]:
grid_point_Weather_data

Date         Time    Id   Longitude   Latitude  Tempreture  \
0     2023-11-28  22:53:55.60     1  143.251973 -35.083201       16.92   
1     2023-11-28  22:53:55.60     2  143.251973 -35.084201       16.92   
2     2023-11-28  22:53:55.60     3  143.251973 -35.085201       16.92   
3     2023-11-28  22:53:55.60     4  143.251973 -35.086201       16.92   
4     2023-11-28  22:53:56.49     5  143.251973 -35.087201       16.93   
...          ...          ...   ...         ...        ...         ...   
3955  2023-11-28  23:11:45.54  3956  143.316973 -35.138201       16.83   
3956  2023-11-28  23:11:46.54  3957  143.316973 -35.139201       16.83   
3957  2023-11-28  23:11:46.54  3958  143.316973 -35.140201       16.83   
3958  2023-11-28  23:11:46.54  3959  143.316973 -35.141201       16.83   
3959  2023-11-28  23:11:46.54  3960  143.316973 -35.142201       16.83   

      Humidity  Wind speed  Weather ID Weather ID group  \
0           89        5.48         500             Rain   
1           89        5.48         500             Rain   
2           89        5.48         500             Rain   
3           89        5.48         500             Rain   
4           89        5.45         500             Rain   
...        ...         ...         ...              ...   
3955        90        5.46         500             Rain   
3956        90        5.46         500             Rain   
3957        90        5.46         500             Rain   
3958        90        5.46         500             Rain   
3959        90        5.46         500             Rain   

     Weather ID description             Sunrise              Sunset  \
0                light rain 2023-11-28 05:07:11 2023-11-28 19:22:34   
1                light rain 2023-11-28 05:07:11 2023-11-28 19:22:34   
2                light rain 2023-11-28 05:07:11 2023-11-28 19:22:34   
3                light rain 2023-11-28 05:07:11 2023-11-28 19:22:34   
4                light rain 2023-11-28 05:07:11 2023-11-28 19:22:35   
...                     ...                 ...                 ...   
3955             light rain 2023-11-28 05:06:48 2023-11-28 19:22:26   
3956             light rain 2023-11-28 05:06:48 2023-11-28 19:22:26   
3957             light rain 2023-11-28 05:06:48 2023-11-28 19:22:26   
3958             light rain 2023-11-28 05:06:48 2023-11-28 19:22:26   
3959             light rain 2023-11-28 05:06:48 2023-11-28 19:22:26   

      Solar activation sconds  
0                       51323  
1                       51323  
2                       51323  
3                       51323  
4                       51324  
...                       ...  
3955                    51338  
3956                    51338  
3957                    51338  
3958                    51338  
3959                    51338  

[3960 rows x 14 columns]

In [23]:
grid_point_Weather_data.isnull().sum()

Date                       0
Time                       0
Id                         0
Longitude                  0
Latitude                   0
Tempreture                 0
Humidity                   0
Wind speed                 0
Weather ID                 0
Weather ID group           0
Weather ID description     0
Sunrise                    0
Sunset                     0
Solar activation sconds    0
dtype: int64

In [24]:
grid_point_Weather_data.to_csv("./results/csv/weather_dataset.csv", index=False)

### Testing  code

In [6]:
lat  = latitudes[0]
long = longitudes[0]

In [7]:
url = "https://api.openweathermap.org/data/2.5/weather?lat={}&lon={}&appid=8ee842d65cf08ec205365865e3d53348&units=metric".format(lat,long)

In [8]:
res = requests.get(url)
data = res.json()

In [9]:
data

{'coord': {'lon': 143.252, 'lat': -35.0832},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 16.58,
  'feels_like': 16.64,
  'temp_min': 16.58,
  'temp_max': 16.58,
  'pressure': 1004,
  'humidity': 90,
  'sea_level': 1004,
  'grnd_level': 996},
 'visibility': 10000,
 'wind': {'speed': 5.01, 'deg': 161, 'gust': 9.17},
 'clouds': {'all': 85},
 'dt': 1701188583,
 'sys': {'country': 'AU', 'sunrise': 1701198415, 'sunset': 1701249811},
 'timezone': 39600,
 'id': 2153292,
 'name': 'Piangil',
 'cod': 200}

In [ ]:
data["wind"]["speed"]

In [ ]:
data["weather"][0]["main"]